신경망: 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성<br>
torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공<br>
PyTorch의 모든 모듈은 nn.Module의 하위 클래스(subclass)

In [1]:
!nvidia-smi

Thu Mar 10 05:28:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    24W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device) # 채널 1, 입력 28*28 텐서를 0에서 1사이의 숫자로 균등하게 생성
logits = model(X)   # Logit: 마지막 노드에서 아무런 Activation Function을 거치지 않은 값
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


https://jins-sw.tistory.com/38

In [8]:
logits

tensor([[-0.0169, -0.0429,  0.0388,  0.1726,  0.0697, -0.0256,  0.0940, -0.0200,
          0.0240,  0.0036]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [10]:
pred_probab

tensor([[0.0953, 0.0928, 0.1007, 0.1151, 0.1039, 0.0944, 0.1064, 0.0950, 0.0992,
         0.0972]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [11]:
torch.sum(pred_probab)

tensor(1., device='cuda:0', grad_fn=<SumBackward0>)

In [12]:
y_pred

tensor([3], device='cuda:0')

모델 계층(Layer)  
FashionMNIST 모델의 계층<br>
28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지

In [13]:
input_image = torch.rand(3,28,28)
input_image.size()

torch.Size([3, 28, 28])

nn.Flatten  
nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환  
(dim=0의 미니배치 차원은 유지)

In [15]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
flat_image.size()

torch.Size([3, 784])

nn.Linear  
선형 계층 은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈

In [19]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
hidden1.size()

torch.Size([3, 20])

nn.ReLU  
비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping) 형성<br>
비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity) 을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 도움

In [20]:
print(f"Before ReLU: {hidden1[0]}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1[0]}")

Before ReLU: tensor([-0.0045, -0.5127, -0.4287, -0.4280, -0.7019,  0.1051,  0.7310, -0.1336,
         0.5023,  0.2341,  0.4704,  0.0568, -0.2057, -0.0355,  0.0036, -0.0744,
        -0.0768,  0.2408, -0.0957,  0.4796], grad_fn=<SelectBackward0>)


After ReLU: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1051, 0.7310, 0.0000, 0.5023,
        0.2341, 0.4704, 0.0568, 0.0000, 0.0000, 0.0036, 0.0000, 0.0000, 0.2408,
        0.0000, 0.4796], grad_fn=<SelectBackward0>)


nn.Sequential  
nn.Sequential 은 순서를 갖는 모듈의 컨테이너  
데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달  
순차 컨테이너(sequential container)를 사용하여 아래의 seq_modules 와 같은 신경망을 빠르게 생성 가능

In [21]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax  
신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits 를 반환  
logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)  
dim 매개변수는 값의 합이 1이 되는 차원

In [22]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [27]:
pred_probab

tensor([[0.1249, 0.1176, 0.0874, 0.1150, 0.0789, 0.0958, 0.1144, 0.0838, 0.0942,
         0.0881],
        [0.1016, 0.1338, 0.0891, 0.1057, 0.0662, 0.1111, 0.1036, 0.0959, 0.1048,
         0.0882],
        [0.1090, 0.1193, 0.0933, 0.1069, 0.0714, 0.1035, 0.1039, 0.0987, 0.0987,
         0.0952]], grad_fn=<SoftmaxBackward0>)

모델 매개변수  
신경망 내부의 많은 계층들은 매개변수화(parameterize) 됨  
즉, 학습 중에 최적화되는 가중치와 편향과 연관  
nn.Module 을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)  
모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근 가능

In [24]:
print("Model structure: ", model)

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [25]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0329,  0.0194,  0.0218,  ...,  0.0087,  0.0010,  0.0119],
        [-0.0281,  0.0337,  0.0162,  ..., -0.0241,  0.0355, -0.0338]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0329, -0.0190], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-3.5301e-02,  1.2454e-02, -2.0082e-02,  ..., -3.3867e-02,
         -1.9725e-05,  3.6817e-02],
        [ 2.2278e-02, -3.7010e-02, -8.0826e-03,  ..., -1.2653e-02,
         -1.3790e-02, -3.2078e-02]], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([0.0135, 0.0161], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[-0.0070,  0.0011, -0.0224,  ...,  0.005